In [0]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score, \
                            precision_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc  
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
rand_seed = 123
random.seed(rand_seed)
np.random.seed(rand_seed)


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_xg = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172_xg_90.csv')
train_xg = train_xg.drop(['Unnamed: 0','bidprice', 'payprice'], axis = 1)

In [0]:
train_xg.head()

,hour,slotprice,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,adexchange_1.0,...,slotsize_200*200,slotsize_300*250,slotsize_336*280,slotsize_360*300,slotsize_640*90,slotsize_728*90,slotsize_760*90,slotsize_910*90,slotsize_950*90,slotsize_960*90
0,13,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,23,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,22,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,12,20,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
train_rf = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172_rf.csv')
train_rf = train_rf.drop(['Unnamed: 0','bidprice','payprice'], axis = 1)

In [0]:
train_rf.head()

,hour,slotprice,weekday_6,slotvisibility_FirstView,slotformat_5,domain1,domain2,domain3,domain4,domain5,...,keypage3,usertag_10006,usertag_10063,usertag_10031,usertag_11278,usertag_10684,usertag_13042,os_android,os_windows,browser_safari
0,13,0,0,1,0,5.0,1.0,-3.0,4.0,0.0,...,-1.0,1,0,0,0,0,0,0,1,0
1,1,0,0,0,0,2.0,-4.0,1.0,1.0,0.0,...,2.0,0,1,0,0,0,0,0,1,0
2,23,0,0,1,0,5.0,-2.0,-5.0,-1.0,0.0,...,-1.0,0,0,0,0,0,0,0,1,0
3,22,0,0,0,0,1.0,-2.0,2.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,1,0
4,12,20,0,0,0,1.0,-2.0,1.0,0.0,1.0,...,2.0,0,0,0,0,0,0,0,1,0


In [0]:
val_xg = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/val_1793_7172_xg_90.csv')
val_xg = val_xg.drop(['Unnamed: 0','bidprice','payprice'], axis = 1)

In [5]:
val_xg.head()

,hour,slotprice,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,adexchange_1.0,...,slotsize_200*200,slotsize_300*250,slotsize_336*280,slotsize_360*300,slotsize_640*90,slotsize_728*90,slotsize_760*90,slotsize_910*90,slotsize_950*90,slotsize_960*90
0,20,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,21,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,8,5,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,15,5,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,18,5,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [0]:
val_rf = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/val_1793_7172_rf.csv')
val_rf = val_rf.drop(['Unnamed: 0','bidprice','payprice'], axis = 1)

In [0]:
val_rf.head()

,hour,slotprice,weekday_6,slotvisibility_FirstView,slotformat_5,domain1,domain2,domain3,domain4,domain5,...,keypage3,usertag_10006,usertag_10063,usertag_10031,usertag_11278,usertag_10684,usertag_13042,os_android,os_windows,browser_safari
0,20,0,0,0,0,-2.0,-2.0,0.0,-1.0,0.0,...,0.0,0,0,0,0,0,0,0,1,0
1,21,0,0,0,0,0.0,-2.0,1.0,1.0,0.0,...,2.0,1,1,0,0,0,0,0,1,0
2,8,5,0,0,0,0.0,-2.0,0.0,-2.0,0.0,...,1.0,1,1,0,0,0,0,0,1,0
3,15,5,0,0,0,-1.0,1.0,-1.0,2.0,-1.0,...,1.0,1,1,0,0,0,0,0,1,0
4,18,5,0,1,0,-2.0,1.0,3.0,2.0,0.0,...,9.0,1,1,0,0,0,0,0,1,0


In [0]:
xg_columns = list(train_xg.columns)
# rf_columns = list(train_rf.columns)

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_TEST_v1.csv')

In [0]:
test_xg = test[xg_columns]

In [0]:
test_xg.to_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/test_xg.csv', sep = ',')

In [0]:
test_rf = test[rf_columns]

In [0]:
test_rf.to_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/test_rf.csv', sep = ',')

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172.csv')

In [0]:
validation = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_VALIDATION_v1.csv')

In [0]:
train_click = train['click']

In [0]:
validation_click = validation['click']

In [0]:
validation_bidid = validation['bidid']

In [0]:
validation_payprice = validation['payprice']

In [0]:
del train
del validation

In [0]:
def classifier(model,  x_train, y_train, x_val, y_val):
        # Fit classifier to training data
    % time model.fit(x_train, y_train)
        # Predict on validation set
    preds = model.predict(x_val)

        # Predicted probabilities
    pred_proba = model.predict_proba(x_val)
    
    pred_proba_click = [p[1] for p in pred_proba]
  
        # Evaluate performance
    print("\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))  #把0分为0 的个数，0-1，1-0，1-1
    print("\n")
    
        # The most important measure is TP/FP rate for the clicked class
    kpi = precision_score(y_val, preds)
    print("Click Precision: {}".format(kpi))
    
    accuracy = accuracy_score(y_val, preds)
    print("\n")
    print("Prediction accuracy: {}".format(accuracy))
    print("\n")
    
    recall = recall_score(y_val, preds)
    print("Recall score: {}".format(recall))
    print("\n")   
    
    roc_auc = roc_auc_score(y_val, preds)
    print("ROC and AUC Score: {}".format(roc_auc))
    print("\n")   
    
    f1 = f1_score(y_val,preds)
    print("f1 score: {}".format(f1))
    print("\n") 
    
    return  pred_proba_click

In [0]:
def recalibrate(ctr):
  w = 7172/(2430981-1793)
  pctr = ctr/(ctr + (1- ctr) / w)
  
  return pctr

In [0]:
def non_linear(ctr, validation_payprice):
#   range_all = np.arange(1, 200, 10)
  range_all = [140]
  lamda_list = [0.000003, 0.000002995]
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  for c in range_all:
    for lamda in lamda_list:
      
      current_bid = 1.3 * c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))    

      budget = 6250 * 1000
      clicks = 0
      impressions = 0
    
      for j in range(len(current_bid)):
        payprice = validation_payprice[j]
        if current_bid[j] > payprice:
          budget = budget - payprice
          if budget >= 0:
            impressions += 1
            if(validation_click[j] == 1):
              clicks += 1
          else:
            budget = budget + payprice
            break
          
      click_list.append(clicks)
      budget_list.append(budget)
      impression_list.append(impressions)
      bid_price.append(current_bid)
#     print("\n")
      print('C = {}, lamda = {}, clicks = {}, impression = {}, budget = {}'.format(c, lamda, clicks, impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
#   c_optimal = range_all[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  lamda_optimal = lamda_list[index_click]
  print('Maxium click is {}, when lamda = {}, lamda = , budget = {}, impression = {}'.format(max_click, lamda_optimal, lamda_optimal, budget_max, impression_max))

  return  bid_price_final

In [0]:
def linear(ctr, validation):
  
#   w = 7172/(2430981-1793)
#   pctr = ctr/(ctr + (1-ctr) / w)
#   avgctr = 1793/2430981
#   avgctr_2 = pctr.mean()
  
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  base_list = []
  
  for base in range(100,130,1):
    current_bid = base * ctr  / ctr.mean()
    budget = 6250*1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation_payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation_click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    base_list.append(base)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base, clicks,  impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_list[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return bid_price_final

In [0]:
def linear_2(ctr, validation):
  
  w = 7172/(2430981-1793)
  pctr = ctr/(ctr + (1-ctr) / w)
  avgctr = 1793/2430981
  avgctr_2 = pctr.mean()
  
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  base_list = []
  for base in range(40,90,1):
    current_bid = base * pctr  / avgctr
    budget = 6250*1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation_payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation_click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    base_list.append(base)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base, clicks,  impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_list[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return bid_price_final

In [0]:
def linear_square(ctr, validation):
  
#   w = 7172/(2430981-1793)
#   pctr = ctr/(ctr + (1-ctr) / w)
#   avgctr = 1793/2430981
#   avgctr_2 = pctr.mean()
  
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  base_list = []
  for base in range(60,80,1):
    current_bid = base * (ctr  / ctr.mean()) ** 1.95
    budget = 6250*1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation_payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation_click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    base_list.append(list)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base, clicks,  impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_list[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return bid_price_final

# Tune parameter

In [0]:

rf_grid = {
           "n_estimators": [120, 300, 500, 800, 1000, 1200],
           "max_depth": [5,8,15]
          }

xg_grid = {
            "n_estimators": [300, 500, 1000],
            "max_depth": [4,8,15,20],
            "learning_rate": [0.001, 0.01, 0.1]
          }

In [0]:
def tune_parameter(model, grid, x_train, y_train, x_val, y_val):
   
    grid = GridSearchCV(model, 
                        param_grid = grid, 
                        scoring = "f1", 
                        cv = StratifiedKFold(n_splits = 3, shuffle = True, random_state = rand_seed)
                        )
    # Fit grid on train data
    grid.fit(x_train, y_train)
    
#     # Store best model
    best_model = grid.best_estimator_
    print("\n")
    print("Best parameters: {}".format(grid.best_params_))
    
    # Predict on validation set
    preds = grid.predict(x_val)
    
#     # Predicted probabilities
#     pred_proba = grid.predict_proba(x_val)
#     pred_proba_click = [p[1] for p in pred_proba]
    
    # Evaluate performance
    print("\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))  #把0分为0 的个数，0-1，1-0，1-1
    print("\n")
    
    kpi = precision_score(y_val, preds)
    print("Click Precision: {}".format(kpi))
    
    accuracy = accuracy_score(y_val, preds)
    print("\n")
    print("Prediction accuracy: {}".format(accuracy))
    print("\n")
    
    recall = recall_score(y_val, preds)
    print("Recall score: {}".format(recall))
    print("\n")   
    
    roc_auc = roc_auc_score(y_val, preds)
    print("ROC and AUC Score: {}".format(roc_auc))
    print("\n")   
   

# XGboost

In [0]:
#Tune parameter
tune_parameter(XGBClassifier(random_state = rand_seed), xg_grid, train_xg, train_click, val_xg, validation_click)



Best parameters: {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 500}


Confusion Matrix:
[[296569   7154]
 [    91    111]]


Click Precision: 0.015278733654507914


Prediction accuracy: 0.9761618820432673


Recall score: 0.5495049504950495


ROC and AUC Score: 0.7629752966999667




In [24]:
ctr_xg = classifier(XGBClassifier(n_estimators = 500, max_depth = 8, learning_rate = 0.01, random_state = rand_seed), 
                    train_xg,train_click, val_xg, validation_click)

CPU times: user 26.3 s, sys: 68 ms, total: 26.4 s
Wall time: 26.4 s


Confusion Matrix:
[[296569   7154]
 [    91    111]]


Click Precision: 0.015278733654507914


Prediction accuracy: 0.9761618820432673


Recall score: 0.5495049504950495


ROC and AUC Score: 0.7629752966999667


f1 score: 0.029730815588589796




In [0]:
'''
CPU times: user 25.5 s, sys: 72 ms, total: 25.6 s
Wall time: 25.6 s


Confusion Matrix:
[[296569   7154]
 [    91    111]]


Click Precision: 0.015278733654507914


Prediction accuracy: 0.9761618820432673


Recall score: 0.5495049504950495


ROC and AUC Score: 0.7629752966999667


f1 score: 0.029730815588589796
'''

In [0]:
pctr_xg = recalibrate(np.array(ctr_xg))

In [0]:
bidprice_val_xg = linear(np.array(ctr_xg), validation_payprice) #base = 75 click = 161

In [0]:
bidprice_val_xg_2 = linear_2(np.array(ctr_xg), validation_payprice) #base = 138  163

In [0]:
bidprice_val_xg_3 = linear_square(np.array(ctr_xg), validation_payprice) #base = 63, click = 163

In [0]:
'''
**1.9
65 164

**1.95
base = 64, clicks = 164, impression = 107358, budget = 18394

**1.8
66 164 6

**1.75
base = 66, clicks = 164, impression = 111420, budget = 37705
base = 67, clicks = 162, impression = 111856, budget = 1

**2.1
base = 61, clicks = 163, impression = 103541, budget = 93649
base = 62, clicks = 163, impression = 104432, budget = 33592
base = 63, clicks = 161, impression = 104794, budget = 4

**2.2
base = 60, clicks = 163, impression = 102098, budget = 79375
base = 61, clicks = 163, impression = 102963, budget = 19649
base = 62, clicks = 161, impression = 103176, budget = 220
'''

In [0]:
bidprice_val_xg_4 = linear_square(np.array(pctr_xg), validation_payprice) #base = 134 click =163 pctr.mean()

In [0]:
bidprice_val_xg_4 = linear_square(np.array(pctr_xg), validation_payprice) #base = 222 click =163 avgctr

# Test


In [0]:
test_xg = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/test_xg.csv')

In [0]:
test_xg = test_xg.drop('Unnamed: 0', axis = 1)

In [0]:
group = pd.read_csv('/content/gdrive/My Drive/we_data/Group_xx.csv')

In [0]:
list(test_xg)

In [0]:
list(train_xg) == list(test_xg)

True

In [0]:
# test_columns = list(test_xg.columns)
# test_xg = test_xg.as_matrix(columns = test_columns)

In [0]:
# test = test.drop(columns = ['bidid', 'userid'])

model = XGBClassifier(n_estimators = 500, max_depth = 8, learning_rate = 0.01, random_state = rand_seed).fit(train_xg, train_click)
preds = model.predict(test_xg)
pred_proba = model.predict_proba(test_xg)

In [0]:
pred_proba = pred_proba[:,1]

In [0]:
pctr_xg = recalibrate(pred_proba)

In [0]:
avgctr

0.0007375623256619447

In [0]:
pctr_xg.mean()

0.00057886395

In [0]:
avgctr = 1793/2430981
# dont forget change the base
current_bid_xg = 64 * (pred_proba / pred_proba.mean()) ** 1.95

In [73]:
len(current_bid_xg)

303375

In [0]:
group['bidprice'] = current_bid_xg

In [75]:
group.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,38.107269
1,29167d4caa719788b5a342dbaa25151d53121f80,317.019226
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,7.017878
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,107.642334
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,60.306297


In [0]:
group.to_csv('/content/gdrive/My Drive/we_data/SUBMISSION_BIDS/XG_1.95_.csv')

In [0]:
group.sort_values('bidprice', ascending = False)